In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string

In [3]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
#bn = BinanceMarket("BTCBUSD")
#bn.download(10)

rbot.init_debug_log()

back_tester = BackTester("BN", "BTCBUSD", False)

print("start")
r = back_tester.run(Agent())
print("end")

print(r)



start
2022-12-05T00:41:20.352Z DEBUG [rbot::sim::back] want on tick  false
2022-12-05T00:41:20.353Z DEBUG [rbot::sim::back] want on clock true
2022-12-05T00:41:20.353Z DEBUG [rbot::sim::back] want on event true
2022-12-05T00:41:20.353Z DEBUG [rbot::sim::back] clock interval 7200
2022-12-05T00:41:20.362Z DEBUG [rbot::db::sqlite] Database open path = /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db
2022-12-05T00:41:20.364Z DEBUG [rbot::db::sqlite] Database open path = /Users/takeoyasukawa/Library/Application Support/net.takibi.rbot/DB/BN/BTCBUSD.db
2022-12-05T00:41:20.364Z DEBUG [rbot::db::sqlite] cache update all 2022-11-11T12:00:00.000000 -> 2022-11-12T00:00:00.928000
2022-12-05T00:41:20.364Z DEBUG [rbot::db::sqlite] create iter 0 microsec
2022-12-05T00:41:20.365Z DEBUG [rbot::db::df] ohlcv_df, from=2022-11-11T12:00:00.000000 / to=2022-11-12T00:00:00.928000
2022-12-05T00:41:20.365Z DEBUG [rbot::db::df] Select from 2022-11-11T12:00:00.000000 -> 2022-11-1

In [6]:
import rbot
df = rbot.result_to_df(r)

In [9]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_home_size,open_foreign_size,close_price,close_home_size,close_foreign_size,order_price,order_home_size,order_foreign_size,profit,fee,total_profit,message,sum_profit
create_time,,,,,,,,,,,,,,,,,,,,
2022-11-13 07:59:59.881000+00:00,2022-11-13 08:00:00.355000+00:00,0000-0001,0,OrderSide.Buy,True,OrderStatus.OpenPosition,16746.45,0.01,167.4645,0.00,0.00,0.0000,16746.45,0.01,167.4645,0.0000,0.016746,-0.016746,Open Long,-0.016746
2022-11-14 07:59:59.985000+00:00,2022-11-14 08:00:04.540000+00:00,0000-0002,0,OrderSide.Sell,True,OrderStatus.ClosePosition,16746.45,0.01,167.4645,16803.44,0.01,168.0344,16803.44,0.01,168.0344,0.5699,0.016803,0.553097,Doten Short,0.536350
2022-11-14 07:59:59.985000+00:00,2022-11-14 08:00:04.540000+00:00,0000-0002,1,OrderSide.Sell,True,OrderStatus.OpenPosition,16803.44,0.01,168.0344,0.00,0.00,0.0000,16803.44,0.01,168.0344,0.0000,0.016803,-0.016803,Doten Short,0.519547
2022-11-15 19:59:59.986000+00:00,2022-11-15 20:00:00.391000+00:00,0000-0003,0,OrderSide.Buy,True,OrderStatus.ClosePosition,16803.44,0.01,168.0344,16841.52,0.01,168.4152,16841.52,0.01,168.4152,-0.3808,0.016842,-0.397642,doten Long,0.121905
2022-11-15 19:59:59.986000+00:00,2022-11-15 20:00:00.391000+00:00,0000-0003,1,OrderSide.Buy,True,OrderStatus.OpenPosition,16841.52,0.01,168.4152,0.00,0.00,0.0000,16841.52,0.01,168.4152,0.0000,0.016842,-0.016842,doten Long,0.105064
2022-11-17 15:59:59.975000+00:00,2022-11-17 16:00:02.380000+00:00,0000-0004,0,OrderSide.Sell,True,OrderStatus.ClosePosition,16841.52,0.01,168.4152,16525.63,0.01,165.2563,16525.63,0.01,165.2563,-3.1589,0.016526,-3.175426,Doten Short,-3.070362
2022-11-17 15:59:59.975000+00:00,2022-11-17 16:00:02.380000+00:00,0000-0004,1,OrderSide.Sell,True,OrderStatus.OpenPosition,16525.63,0.01,165.2563,0.00,0.00,0.0000,16525.63,0.01,165.2563,0.0000,0.016526,-0.016526,Doten Short,-3.086888
2022-11-18 15:59:59.371000+00:00,2022-11-18 16:00:06.661000+00:00,0000-0005,0,OrderSide.Buy,True,OrderStatus.ClosePosition,16525.63,0.01,165.2563,16679.83,0.01,166.7983,16679.83,0.01,166.7983,-1.5420,0.016680,-1.558680,doten Long,-4.645567
2022-11-18 15:59:59.371000+00:00,2022-11-18 16:00:06.661000+00:00,0000-0005,1,OrderSide.Buy,True,OrderStatus.OpenPosition,16679.83,0.01,166.7983,0.00,0.00,0.0000,16679.83,0.01,166.7983,0.0000,0.016680,-0.016680,doten Long,-4.662247


In [8]:
df['total_profit'].sum()

-10.834932030000163

In [21]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.order_home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, 
          "open_price": open_price, "open_home_size": open_home_size, "open_foreign_size": open_foreign_size, 
          "close_price": close_price, "close_home_size": close_home_size, "close_foreign_size": close_foreign_size,
          "order_price": order_price, "order_home_size": order_home_size, "order_foreign_size": order_foreign_size,
          "profit": profit, "fee": fee,
          "total_profit": total_profit, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_side", "post_only",
             "create_time", "status", "open_price", "open_home_size", "open_foreign_size", 
             "close_price", "close_home_size", "close_foreign_size",             
             "order_price", "order_home_size", "order_foreign_size",
             "profit", "fee", "total_profit", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="us")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="us")
    df["sum_profit"] = df["total_profit"].cumsum()
    # df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)
    
    return df

In [22]:
df = result_to_df(r)
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_home_size,open_foreign_size,close_price,close_home_size,close_foreign_size,order_price,order_home_size,order_foreign_size,profit,fee,total_profit,message,sum_profit
create_time,,,,,,,,,,,,,,,,,,,,
2022-11-13 07:59:59.881000+00:00,2022-11-13 08:00:00.355000+00:00,0000-0001,0,OrderSide.Buy,True,OrderStatus.OpenPosition,16746.45,0.01,167.4645,0.00,0.00,0.0000,16746.45,0.01,167.4645,0.0000,0.016746,-0.016746,Open Long,-0.016746
2022-11-14 07:59:59.985000+00:00,2022-11-14 08:00:04.540000+00:00,0000-0002,0,OrderSide.Sell,True,OrderStatus.ClosePosition,0.00,0.00,0.0000,16803.44,0.01,168.0344,16803.44,0.01,168.0344,168.0344,0.016803,168.017597,Doten Short,168.000850
2022-11-14 07:59:59.985000+00:00,2022-11-14 08:00:04.540000+00:00,0000-0002,1,OrderSide.Sell,True,OrderStatus.OpenPosition,16803.44,0.01,168.0344,0.00,0.00,0.0000,16803.44,0.01,168.0344,0.0000,0.016803,-0.016803,Doten Short,167.984047
2022-11-15 19:59:59.986000+00:00,2022-11-15 20:00:00.391000+00:00,0000-0003,0,OrderSide.Buy,True,OrderStatus.ClosePosition,0.00,0.00,0.0000,16841.52,0.01,168.4152,16841.52,0.01,168.4152,-168.4152,0.016842,-168.432042,doten Long,-0.447995
2022-11-15 19:59:59.986000+00:00,2022-11-15 20:00:00.391000+00:00,0000-0003,1,OrderSide.Buy,True,OrderStatus.OpenPosition,16841.52,0.01,168.4152,0.00,0.00,0.0000,16841.52,0.01,168.4152,0.0000,0.016842,-0.016842,doten Long,-0.464836
2022-11-17 15:59:59.975000+00:00,2022-11-17 16:00:02.380000+00:00,0000-0004,0,OrderSide.Sell,True,OrderStatus.ClosePosition,0.00,0.00,0.0000,16525.63,0.01,165.2563,16525.63,0.01,165.2563,165.2563,0.016526,165.239774,Doten Short,164.774938
2022-11-17 15:59:59.975000+00:00,2022-11-17 16:00:02.380000+00:00,0000-0004,1,OrderSide.Sell,True,OrderStatus.OpenPosition,16525.63,0.01,165.2563,0.00,0.00,0.0000,16525.63,0.01,165.2563,0.0000,0.016526,-0.016526,Doten Short,164.758412
2022-11-18 15:59:59.371000+00:00,2022-11-18 16:00:06.661000+00:00,0000-0005,0,OrderSide.Buy,True,OrderStatus.ClosePosition,0.00,0.00,0.0000,16679.83,0.01,166.7983,16679.83,0.01,166.7983,-166.7983,0.016680,-166.814980,doten Long,-2.056567
2022-11-18 15:59:59.371000+00:00,2022-11-18 16:00:06.661000+00:00,0000-0005,1,OrderSide.Buy,True,OrderStatus.OpenPosition,16679.83,0.01,166.7983,0.00,0.00,0.0000,16679.83,0.01,166.7983,0.0000,0.016680,-0.016680,doten Long,-2.073247


In [4]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [6]:
bn

In [4]:
print(bn.__str__())

NameError: name 'bn' is not defined

In [5]:
bn.info()

'{"start": 2022-11-12T00:00:00.019000, "end": 2022-11-29T23:59:59.728000}'

In [6]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

2022-11-30T11:39:59.653Z DEBUG [rbot::db::sqlite] cache update all 1970-01-01T00:00:00.000000 -> 2022-11-27T23:59:59.839000
2022-11-30T11:39:59.654Z DEBUG [rbot::db::sqlite] create iter 0 microsec
2022-11-30T11:40:20.856Z DEBUG [rbot::db::df] ohlcv_df, from=1970-01-01T00:00:00.000000 / to=2022-11-27T23:59:59.839000
2022-11-30T11:40:20.856Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-27T23:59:59.839000
2022-11-30T11:40:21.543Z DEBUG [rbot::db::df] ohlcv_df, from=1970-01-01T00:00:00.000000 / to=2022-11-27T23:59:59.839000
2022-11-30T11:40:21.543Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-27T23:59:59.839000


,order_side,open,high,low,close,vol,count,start_time,end_time
timestamp,,,,,,,,,
2022-11-11 23:52:05+00:00,Sell,17032.26,17059.89,17005.94,17059.54,120.60122,2818.0,2022-11-12 00:00:00.019000064+00:00,2022-11-12 00:02:05.960000+00:00
2022-11-11 23:52:05+00:00,Buy,17030.40,17056.03,17004.65,17055.78,109.67540,2479.0,2022-11-12 00:00:00.033999872+00:00,2022-11-12 00:02:05.991000064+00:00
2022-11-12 00:02:06+00:00,Sell,17055.39,17085.64,16977.80,16980.96,374.37585,8715.0,2022-11-12 00:02:06.059000064+00:00,2022-11-12 00:12:06.992000+00:00
2022-11-12 00:02:06+00:00,Buy,17055.54,17081.13,16975.27,16977.66,355.86684,9228.0,2022-11-12 00:02:06.014000128+00:00,2022-11-12 00:12:06.931000064+00:00
2022-11-12 00:12:07+00:00,Sell,16981.43,16998.97,16924.36,16925.37,306.29826,7107.0,2022-11-12 00:12:07.001999872+00:00,2022-11-12 00:22:07.817999872+00:00
...,...,...,...,...,...,...,...,...,...
2022-11-27 23:30:23+00:00,Buy,16455.00,16461.06,16400.00,16431.88,982.65514,18063.0,2022-11-27 23:30:23.123000064+00:00,2022-11-27 23:40:23.556999936+00:00
2022-11-27 23:40:24+00:00,Sell,16432.11,16451.55,16392.14,16444.27,699.69427,14026.0,2022-11-27 23:40:24.008999936+00:00,2022-11-27 23:50:24.516000+00:00
2022-11-27 23:40:24+00:00,Buy,16432.06,16450.04,16392.14,16442.18,723.56509,15058.0,2022-11-27 23:40:24.507000064+00:00,2022-11-27 23:50:24.792999936+00:00


In [7]:
bn.ohlcv(0, 0, 600)

2022-11-30T11:40:22.187Z DEBUG [rbot::db::sqlite] create iter 1 microsec
2022-11-30T11:40:22.188Z DEBUG [rbot::db::sqlite] load data before cache df1=(0, 4) df2=(36890162, 4)
2022-11-30T11:40:22.196Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-12T00:00:00.019000
2022-11-30T11:40:22.196Z DEBUG [rbot::db::df] merge len 0
2022-11-30T11:40:22.196Z DEBUG [rbot::db::sqlite] cache update diff before 1970-01-01T00:00:00.000000 -> 2022-11-12T00:00:00.000000
2022-11-30T11:40:22.196Z DEBUG [rbot::db::df] ohlcv_df, from=1970-01-01T00:00:00.000000 / to=2022-11-12T00:00:00.000000
2022-11-30T11:40:22.196Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-12T00:00:00.000000
2022-11-30T11:40:22.207Z DEBUG [rbot::db::df] empty ohlcv
2022-11-30T11:40:22.207Z DEBUG [rbot::db::df] Select from 2022-11-12T00:00:00.000000 -> 1970-01-01T00:00:00.000000
2022-11-30T11:40:22.207Z DEBUG [rbot::db::df] Select from 1970-01-01T00:00:00.000000 -> 2022-11-12T00:00:00.0000

,open,high,low,close,vol,count
timestamp,,,,,,
2022-11-12 00:00:00+00:00,17032.26,17085.64,16981.00,16998.97,452.28175,10393.0
2022-11-12 00:00:00+00:00,17030.40,17081.13,16979.26,16997.03,412.54882,10365.0
2022-11-12 00:10:00+00:00,16997.44,16999.36,16927.55,16940.34,289.98533,6992.0
2022-11-12 00:10:00+00:00,16997.21,16998.77,16924.57,16938.15,336.32993,8528.0
2022-11-12 00:20:00+00:00,16940.24,16958.49,16883.24,16901.55,400.16489,8312.0
...,...,...,...,...,...,...
2022-11-27 23:30:00+00:00,16462.00,16462.00,16400.00,16438.45,990.39619,18166.0
2022-11-27 23:40:00+00:00,16438.83,16451.55,16392.14,16444.89,692.29492,14117.0
2022-11-27 23:40:00+00:00,16438.11,16450.04,16392.14,16443.54,725.08376,15008.0


In [15]:
def result_to_df(result_list):
    update_time = []
    order_id = []
    order_sub_id = []
    order_side = []
    post_only = []
    create_time = []
    status = []
    open_price = []
    open_home_size = []
    open_foreign_size = []
    close_price = []
    close_home_size = []
    close_foreign_size = []
    order_price = []
    order_home_size = []
    order_foreign_size = []
    profit = []
    fee = []
    total_profit = []
    message = []

    for item in result_list:
        update_time.append(item.update_time)
        order_id.append(item.order_id)
        order_sub_id.append(item.order_sub_id)
        order_side.append(item.order_side)
        post_only.append(item.post_only)
        create_time.append(item.create_time)
        status.append(item.status)
        open_price.append(item.open_price)
        open_home_size.append(item.open_home_size)
        open_foreign_size.append(item.open_foreign_size)
        close_price.append(item.close_price)
        close_home_size.append(item.close_home_size)
        close_foreign_size.append(item.close_foreign_size)
        order_price.append(item.order_price)
        order_home_size.append(item.home_size)
        order_foreign_size.append(item.order_foreign_size)
        profit.append(item.profit)
        fee.append(item.fee)
        total_profit.append(item.total_profit)
        message.append(item.message)

    df = pd.DataFrame(
    data={"update_time": update_time, "order_id": order_id, "sub_id": order_sub_id,
          "order_side": order_side, "post_only": post_only, "create_time": create_time,
          "status":  status, "open_price": open_price, "open_home_size": open_home_size, 
          "open_foreign_size": open_foreign_size, "close_price": close_price,
          "price": price, "size": size, "volume": volume, "profit": profit, "fee": fee,
          "total_profit": total_profit, "pos_change": position_change, "message": message},
    columns=["update_time", "order_id", "sub_id", "order_type", "post_only",
             "create_time", "status", "open_price", "close_price", "price", "size", "volume",
             "profit", "fee", "total_profit", "pos_change", "message"])
    df["update_time"] = pd.to_datetime((df["update_time"]), utc=True, unit="ms")
    df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit="ms")
    df["sum_profit"] = df["total_profit"].cumsum()
    df["sum_pos"] = df["pos_change"].cumsum()
    df = df.set_index("create_time", drop=True)

In [16]:
result_to_df(r)

AttributeError: 'builtins.OrderResult' object has no attribute 'update_time'

In [19]:
r[0].update_time

AttributeError: 'builtins.OrderResult' object has no attribute 'update_time'